# EMG Dataset Training and Inference with TCNN Network

Made by STMicroelectronics


Authors: Fabrizio Maria Aymone, Danilo Pau

contact: danilo.pau@st.com

### Download NinaPRO DB8 Dataset

In [ ]:
!wget --continue https://data.ncl.ac.uk/ndownloader/articles/9577598/versions/1

In [ ]:
!unzip 1
!mkdir ninaproDB8
!unzip \*.zip -d ninaproDB8

## Load Dataset and preprocesing

In [ ]:
import numpy as np
import os
from scipy.io import loadmat
from scipy import signal


linear_transformation_matrix = np.matrix([[0.639, 0.000, 0.000, 0.000, 0.000],
                                                     [0.383, 0.000, 0.000, 0.000, 0.000],
                                                     [0.000, 1.000, 0.000, 0.000, 0.000],
                                                     [-0.639,0.000, 0.000, 0.000, 0.000],
                                                     [0.000, 0.000, 0.400, 0.000, 0.000],
                                                     [0.000, 0.000, 0.600, 0.000, 0.000],
                                                     [0.000, 0.000, 0.000, 0.400, 0.000],
                                                     [0.000, 0.000, 0.000, 0.600, 0.000],
                                                     [0.000, 0.000, 0.000, 0.000, 0.000],
                                                     [0.000, 0.000, 0.000, 0.000,0.1667],
                                                     [0.000, 0.000, 0.000, 0.000,0.3333],
                                                     [0.000, 0.000, 0.000, 0.000, 0.000],
                                                     [0.000, 0.000, 0.000, 0.000,0.1667],
                                                     [0.000, 0.000, 0.000, 0.000,0.3333],
                                                     [0.000, 0.000, 0.000, 0.000, 0.000],
                                                     [0.000, 0.000, 0.000, 0.000, 0.000],
                                                     [-0.19, 0.000, 0.000, 0.000, 0.000],
                                                     [0.000, 0.000, 0.000, 0.000, 0.000]
                                                     ])

from sklearn.preprocessing import StandardScaler

def read_emg(emg_path):
    # extract nth channel EMG, bandpass,  down-sampling, normalize
    #b, a = signal.butter(4, [10,500], 'bp',fs=2000) #bandpass = signal.butter(4, [20,500], 'bandpass',output='sos',fs=2000)
    emg_data = loadmat(emg_path)

    X = emg_data.get('emg')
    #X = signal.filtfilt(b,a,X,axis=0)
    scale = 0.011*2/ (2**16-1) #the range of the Delsys Trigno is +-11 mV and the precision is int16
    X = np.round(X/scale).astype(np.int16)
    
    y = emg_data.get('glove') @ linear_transformation_matrix
   
    y = y.astype('float32')

    stimulus = emg_data.get('restimulus')
    repetition = emg_data.get('rerepetition')

    return X, y, stimulus, repetition

In [ ]:
import tensorflow as tf
from tensorflow import keras
from itertools import chain
import gc

class CustomWindowGenerator(keras.utils.Sequence):

    def __init__(self, X, y, index_array=None, window_size=256, batch_size=64, slide=50, shuffle=True, pass_array=False):
        
        # X and y are lists of datasets e.g. if we have two datasets (X1, y1) and (X2, y2), X is [X1, X2] and y is [y1, y2]
        
        self.X = X
        self.y = y
        self.w = window_size
        self.batch_size = batch_size
        self.slide = slide
        self.shuffle = shuffle
        self.index_array= list(chain(*[zip(np.repeat(idx, len(x)), np.arange(0, (len(x)-window_size+1), slide)) for idx, x in enumerate(X)])) if pass_array==False else index_array
        if self.shuffle:
            np.random.shuffle(self.index_array)
        #self.epoch=1

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.index_array)
        gc.collect()
        keras.backend.clear_session()
    
    def __getitem__(self, index):

        X_d = []
        y_d = []
        
        for n, i in self.index_array[index*self.batch_size:index*self.batch_size + self.batch_size]:
            
            X_d.append([self.X[n][i:i+self.w]])
            y_d.append(self.y[n][i+self.w-1])
        
        return np.vstack(X_d), np.vstack(y_d)
    
    def __len__(self):
        return len(self.index_array)//self.batch_size

In [ ]:
def prepare_dataset_for_user(s):
    X_train_1, y_train_1, stimulus_1, repetition_1 = read_emg(f"./ninaproDB8/S{s}_E1_A1.mat")
    X_train_2, y_train_2, stimulus_2, repetition_2 = read_emg(f"./ninaproDB8/S{s}_E1_A2.mat")
    
    
    X_train, y_train, stimulus_train, repetition_train = [X_train_1, X_train_2], [y_train_1, y_train_2], [stimulus_1, stimulus_2], [repetition_1, repetition_2]
    
    X_test, y_test, _, _ = read_emg(f"./ninaproDB8/S{s}_E1_A3.mat")
    X_test, y_test = [X_test], [y_test]

    return X_train, y_train, stimulus_train, repetition_train, X_test, y_test
    

In [ ]:
def get_model():

    model = keras.Sequential([

                # BLOCK 1
    
                keras.layers.Input(shape=(256, 16)),
                keras.layers.ZeroPadding1D(padding=((3-1)*2, 0)),
                keras.layers.Conv1D(filters=16, kernel_size=3, padding='valid', strides=1, dilation_rate = 2, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.ZeroPadding1D(padding=((3-1)*2, 0)),
                keras.layers.Conv1D(filters=16, kernel_size=3, padding='valid', strides=1, dilation_rate = 2, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.ZeroPadding1D(padding=((5-1)*1, 0)),
                keras.layers.Conv1D(filters=16, kernel_size=5, padding='valid', strides=1, use_bias=False),
                keras.layers.AveragePooling1D(pool_size=2, strides=2),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                

                # BLOCK 2

                keras.layers.ZeroPadding1D(padding=((3-1)*4, 0)),
                keras.layers.Conv1D(filters=32, kernel_size=3, padding='valid', strides=1, dilation_rate=4, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.ZeroPadding1D(padding=((3-1)*4, 0)),
                keras.layers.Conv1D(filters=32, kernel_size=3, padding='valid', strides=1, dilation_rate=4, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.ZeroPadding1D(padding=((5-1)*1, 0)),
                keras.layers.Conv1D(filters=32, kernel_size=5, padding='valid', strides=2, use_bias=False),
                keras.layers.AveragePooling1D(pool_size=2, strides=2),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),

                # BLOCK 3

                keras.layers.ZeroPadding1D(padding=((3-1)*8, 0)),
                keras.layers.Conv1D(filters=64, kernel_size=3, padding='valid', strides=1, dilation_rate=8, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.ZeroPadding1D(padding=((3-1)*8, 0)),
                keras.layers.Conv1D(filters=64, kernel_size=3, padding='valid', strides=1, dilation_rate=8, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.ZeroPadding1D(padding=((5-1)*1, 0)),
                keras.layers.Conv1D(filters=64, kernel_size=5, padding='valid', strides=4, use_bias=False),
                keras.layers.AveragePooling1D(pool_size=2, strides=2),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),

                # DENSE BLOCK
                keras.layers.Flatten(),
                keras.layers.Dense(4*64, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.Dropout(rate=0.5),
                keras.layers.Dense(32, use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.ReLU(),
                keras.layers.Dropout(rate=0.5),
                keras.layers.Dense(5, use_bias=False)

                ])
    
    return model


In [ ]:
def fit_model(train_gen, val_gen, checkpoint_filepath):

    gc.collect()
    keras.backend.clear_session()
    
    
    model = get_model()
    
    opt = keras.optimizers.Adam(learning_rate=10**(-4))

    model.compile(loss="MeanAbsoluteError", optimizer=opt)
    
    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='min')
    reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
    checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=1, save_freq = "epoch", monitor='val_loss', mode='min')

    model.fit(train_gen, validation_data = val_gen, epochs=50, callbacks=[reduce_lr_loss, checkpoint, earlyStopping])

    return model
    
    
    

In [ ]:
from qkeras import *

def get_qmodel():
    
    # check kernel constraint kernel_constraint=None

    qmodel = keras.Sequential([

        # BLOCK 1

        keras.layers.Input(shape=(256, 16)),
        keras.layers.ZeroPadding1D(padding=((3-1)*2, 0)),
        QConv1D(filters=16, kernel_size=3, padding='valid', strides=1, dilation_rate = 2, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.ZeroPadding1D(padding=((3-1)*2, 0)),
        QConv1D(filters=16, kernel_size=3, padding='valid', strides=1, dilation_rate = 2, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.ZeroPadding1D(padding=(5-1, 0)),
        QConv1D(filters=16, kernel_size=5, padding='valid', strides=1, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.AveragePooling1D(pool_size=2, strides=2),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        
        # BLOCK 2

        keras.layers.ZeroPadding1D(padding=((3-1)*4, 0)),
        QConv1D(filters=32, kernel_size=3, padding='valid', strides=1, dilation_rate=4, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.ZeroPadding1D(padding=((3-1)*4, 0)),
        QConv1D(filters=32, kernel_size=3, padding='valid', strides=1, dilation_rate=4, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.ZeroPadding1D(padding=(5-1, 0)),
        QConv1D(filters=32, kernel_size=5, padding='valid', strides=2, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.AveragePooling1D(pool_size=2, strides=2),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),

        # BLOCK 3

        keras.layers.ZeroPadding1D(padding=((3-1)*8, 0)),
        QConv1D(filters=64, kernel_size=3, padding='valid', strides=1, dilation_rate=8, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.ZeroPadding1D(padding=((3-1)*8, 0)),
        QConv1D(filters=64, kernel_size=3, padding='valid', strides=1, dilation_rate=8, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.ZeroPadding1D(padding=(5-1, 0)),
        QConv1D(filters=64, kernel_size=5, padding='valid', strides=4, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.AveragePooling1D(pool_size=2, strides=2),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),

        # DENSE BLOCK

        keras.layers.Flatten(),
        QDense(4*64, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.Dropout(rate=0.5),
        QDense(32, kernel_quantizer="quantized_bits(8)", use_bias=False),
        keras.layers.BatchNormalization(),
        QActivation("quantized_relu(8)"),
        keras.layers.Dropout(rate=0.5),
        QDense(5, kernel_quantizer="quantized_bits(8)", use_bias=False)
        ])
    
    return qmodel
    


In [ ]:
from qkeras import *

def fit_qmodel(train_gen, val_gen, checkpoint_filepath):

    gc.collect()
    keras.backend.clear_session()
    
    
    model = get_qmodel()
    
    opt = keras.optimizers.Adam(learning_rate=10**(-4))

    model.compile(loss="MeanAbsoluteError", optimizer=opt)
    
    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='min')
    reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
    checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=1, save_freq = "epoch", monitor='val_loss', mode='min')

    model.fit(train_gen, validation_data = val_gen, epochs=50, callbacks=[reduce_lr_loss, checkpoint, earlyStopping])

    return model

In [ ]:
from sklearn.metrics import mean_absolute_error

def get_metrics(predictions, y_test):
    MAE = mean_absolute_error(predictions, np.array(y_test))

    count_10 = 0
    count_15 = 0

    for i in range(0, len(predictions)):
        for j in range(5):
            if np.abs(predictions[i,j]-y_test[i,j]) <= 10:
                count_10+=1
            if np.abs(predictions[i,j]-y_test[i,j]) <= 15:
                count_15+=1
    
    acc_10 = count_10/(len(predictions)*5)
    acc_15 = count_15/(len(predictions)*5)

    print(f"MAE: {MAE}")
    print(f"acc_10: {acc_10}")
    print(f"acc_15: {acc_15}")

    return MAE, acc_10, acc_15

In [ ]:
def get_groups(stimulus):

    a = 0
    prev = 0

    groups = np.zeros(len(stimulus))

    for i in range(len(stimulus)):
        groups[i] = a

        if stimulus[i]==0:
            a+=1

    return groups      

In [ ]:
import sklearn.model_selection

window_size = 256
slide = 1
batch_size = 256

def train_user(s):
    X_train, y_train, stimulus_train, repetition_train, X_test, y_test = prepare_dataset_for_user(s)
    
                                                                           
    # scaler = StandardScaler(with_mean=True, with_std=True, copy=False).fit(np.vstack(X_train))
    # X_train = [scaler.transform(x) for x in X_train]
    # X_test = [scaler.transform(x) for x in X_test]
    

    test_gen = CustomWindowGenerator(X_test, y_test, window_size=256, batch_size=1363, slide=1, shuffle=False, pass_array=False)
    

    gkfold = sklearn.model_selection.StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)


    index_array = np.array(list(chain(*[zip(np.repeat(idx, len(x)), np.arange(0, (len(x)-window_size+1), slide)) for idx, x in enumerate(X_train)])))

    # stimulus for stratified k-folding
    
    stimulus_mock = []
    for idx in range(len(index_array)):
        stimulus_mock.append(stimulus_train[index_array[idx][0]][index_array[idx][1]])
    stimulus_mock = np.vstack(stimulus_mock)

    groups = get_groups(stimulus_mock)

    # balance dataset according to stimulus (stimulus=0 is oversampled)
    #index_array = index_array[np.where(stimulus_mock!=0)[0]]
    #stimulus_mock = stimulus_mock[stimulus_mock!=0]
    
    #over_sampler = imblearn.over_sampling.RandomOverSampler(random_state=42)
    #index_array, stimulus_mock = over_sampler.fit_resample(index_array, stimulus_mock)

    
    fold = 1
    
    for train_idx, val_idx in gkfold.split(X = np.zeros(len(index_array)), y = stimulus_mock, groups=groups):
        
        print(f"\n\n*****************************************Fold: {fold}*****************************************")
        
        train_gen, val_gen = CustomWindowGenerator(X_train, y_train, index_array = index_array[train_idx] ,window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True), \
                           CustomWindowGenerator(X_train, y_train, index_array = index_array[val_idx] ,window_size=window_size, batch_size=batch_size, slide=slide, shuffle=True, pass_array=True)

        #----------------------------------------------- KERAS ----------------------------------------------------------------------------------------------

        keras_models_dir = "keras_models/"
        os.makedirs(keras_models_dir, exist_ok=True)
        mcp_path = keras_models_dir+'keras_k_fold_'+str(fold)+'.h5'        
        
        model = fit_model(train_gen, val_gen, checkpoint_filepath=mcp_path)
        model = keras.models.load_model(mcp_path)

        predictions = model.predict(test_gen)
        MAE, acc_10, acc_15 = get_metrics(predictions, y_test[0][window_size-1:])
        print(f"\n\n*****************************************Keras validation {fold} MAE: {MAE} acc_10: {acc_10}, acc_15: {acc_15} *****************************************")


        #----------------------------------------------- TFLITE ----------------------------------------------------------------------------------------------
        
        
        def representative_data_gen():
                repr_index_array = index_array
                np.random.shuffle(repr_index_array)
                for i in range(100):
                        j = repr_index_array[i]
                        yield [np.expand_dims(X_train[j[0]][j[1]:j[1]+window_size].astype(np.float32), axis=0)]
        
        
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        # Ensure that if any ops can't be quantized, the converter throws an error
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        # Set the input and output tensors to float32 and int8 (APIs added in r2.3)
        converter.inference_input_type = tf.float32
        converter.inference_output_type = tf.int8
        tflite_model_quant = converter.convert()
        
        tflite_models_dir = "tflite_models/"
        os.makedirs(tflite_models_dir, exist_ok=True)
        tflite_model_path = tflite_models_dir+"tflite_k_fold_"+str(fold)+".tflite"
        with open(tflite_model_path, 'wb') as f:
                f.write(tflite_model_quant)

        tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
        input_details = tflite_interpreter.get_input_details()
        output_details = tflite_interpreter.get_output_details()
        tflite_interpreter.resize_tensor_input(input_details[0]['index'], (1363, window_size, 16))
        tflite_interpreter.allocate_tensors()
        scale, zero_point = output_details[0]["quantization"]
    
        tflite_predictions = []

        for X, _ in test_gen:
            tflite_interpreter.set_tensor(input_details[0]['index'], X.astype(np.float32))
            tflite_interpreter.invoke()
            tflite_predictions.append(tflite_interpreter.get_tensor(output_details[0]['index']))
        
        tflite_predictions = np.vstack(tflite_predictions)
        tflite_predictions = (tflite_predictions.astype(np.float32)-zero_point)*scale

        MAE, acc_10, acc_15 = get_metrics(tflite_predictions, y_test[0][window_size-1:])
        print(f"\n\n*****************************************TFlite validation {fold} MAE: {MAE} acc_10: {acc_10}, acc_15: {acc_15} *****************************************")


        #----------------------------------------------- QKERAS ----------------------------------------------------------------------------------------------


        qkeras_models_dir = "qkeras_models/"
        os.makedirs(qkeras_models_dir, exist_ok=True)
        mcp_path = qkeras_models_dir+'qkeras_k_fold_'+str(fold)+'.h5' 
        
        qmodel = fit_qmodel(train_gen, val_gen, checkpoint_filepath=mcp_path)
        
        qmodel.load_weights(mcp_path)
        
        predictions = qmodel.predict(test_gen)
        
        MAE, acc_10, acc_15 = get_metrics(predictions, y_test[0][window_size-1:])
        print(f"\n\n*****************************************QKeras validation {fold} MAE: {MAE} acc_10: {acc_10}, acc_15: {acc_15} *****************************************")
        

        gc.collect()
        keras.backend.clear_session()
        fold+=1

In [ ]:
train_user(1)

## Results

In [ ]:
example = "./keras_models/keras_k_fold_1.h5"
model = keras.models.load_model(example)
model.summary()

In [ ]:
example = "./qkeras_models/qkeras_k_fold_1.h5"
qmodel = get_qmodel()
qmodel.load_weights(example)

In [ ]:
keras_models_dir = "keras_models/"
tflite_models_dir = "tflite_models/"
qkeras_models_dir = "qkeras_models/"



def test_models():

    X_train, X_test, _, _, X_test, y_test = prepare_dataset_for_user(1)
    
    test_gen = CustomWindowGenerator(X_test, y_test, window_size=256, batch_size=1363, slide=1, shuffle=False, pass_array=False) #


    results = {"keras":[], "tflite":[], "qkeras":[]}

    #keras

    model = get_model()
    
    for fold in range(1, 5+1):
        path = keras_models_dir + "keras_k_fold_" + str(fold) + ".h5"
        model.load_weights(path)
        predictions = model.predict(test_gen)
        results["keras"].append(list(get_metrics(predictions, y_test[0][window_size-1:])))
    
    #tflite
    
    for fold in range(1, 5+1):

        path = tflite_models_dir + "tflite_k_fold_" + str(fold) + ".tflite"
        tflite_interpreter = tf.lite.Interpreter(model_path=path)
        input_details = tflite_interpreter.get_input_details()
        output_details = tflite_interpreter.get_output_details()
        scale, zero_point = output_details[0]['quantization']

        tflite_interpreter.resize_tensor_input(input_details[0]['index'], (1363, window_size, 16))
        tflite_interpreter.allocate_tensors()

        predictions = []

        for X, _ in test_gen:
            tflite_interpreter.set_tensor(input_details[0]['index'], X.astype(np.float32))
            tflite_interpreter.invoke()
            predictions.append(tflite_interpreter.get_tensor(output_details[0]['index']))
        
        predictions = np.vstack(predictions)
        predictions = (predictions.astype(np.float32)-zero_point)*scale

        results["tflite"].append(list(get_metrics(predictions, y_test[0][window_size-1:])))
    

    #qkeras
    
    qmodel = get_qmodel()
    
    for fold in range(1, 5+1):
        path = qkeras_models_dir + "qkeras_k_fold_" + str(fold) + ".h5"
        qmodel.load_weights(path)
        predictions = qmodel.predict(test_gen)
        results["qkeras"].append(list(get_metrics(predictions, y_test[0][window_size-1:])))

    return results

In [ ]:
results = test_models()

In [ ]:
import numpy as np
results = {"keras":[], "tflite":[], "qkeras":[]}

results["keras"] = np.load("./results/results_keras.npy")
results["tflite"] = np.load("./results/results_tflite.npy")
results["qkeras"] = np.load("./results/results_qkeras.npy")

In [ ]:
from matplotlib import pyplot as plt


from tabulate import tabulate
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import scienceplots

plt.style.use(['science','ieee', 'no-latex'])


def plot_metrics(results):

    keras_MAE = np.asarray(results["keras"])[:,0]
    tflite_MAE = np.asarray(results["tflite"])[:,0]
    qkeras_MAE = np.asarray(results["qkeras"])[:,0]

    keras_acc_10 = np.asarray(results["keras"])[:,1]
    tflite_acc_10 = np.asarray(results["tflite"])[:,1]
    qkeras_acc_10 = np.asarray(results["qkeras"])[:,1]

    keras_acc_15 = np.asarray(results["keras"])[:,2]
    tflite_acc_15 = np.asarray(results["tflite"])[:,2]
    qkeras_acc_15 = np.asarray(results["qkeras"])[:,2]

    print(qkeras_MAE)

    fig, axs = plt.subplots(1, 3, figsize=(12, 3))
    fig.subplots_adjust(top=0.8)
    x = [0, 1, 2]
    labels = ["Keras", "TFlite", "QKeras"]

    axs[0].errorbar(x, [np.average(keras_MAE), np.average(tflite_MAE), np.average(qkeras_MAE)], yerr=[np.std(keras_MAE), np.std(tflite_MAE), np.std(qkeras_MAE)], capsize=3, ms=3,fmt="r--o", ecolor = "black")
    axs[0].set_xticks(x, labels)
    axs[0].set_title("MAE")

    print(np.average(keras_MAE), np.average(tflite_MAE), np.average(qkeras_MAE),np.std(keras_MAE), np.std(tflite_MAE), np.std(qkeras_MAE))

    axs[1].errorbar(x, [np.average(keras_acc_10), np.average(tflite_acc_10), np.average(qkeras_acc_10)], yerr=[np.std(keras_acc_10), np.std(tflite_acc_10), np.std(qkeras_acc_10)], capsize=3, ms=3,fmt="r--o", ecolor = "black")
    axs[1].set_xticks(x, labels)
    axs[1].set_title("Acc. 10$^\circ$")

    print(np.average(keras_acc_10), np.average(tflite_acc_10), np.average(qkeras_acc_10), np.std(keras_acc_10), np.std(tflite_acc_10), np.std(qkeras_acc_10))

    axs[2].errorbar(x, [np.average(keras_acc_15), np.average(tflite_acc_15), np.average(qkeras_acc_15)], yerr=[np.std(keras_acc_15), np.std(tflite_acc_15), np.std(qkeras_acc_15)], capsize=3, ms=3,fmt="r--o", ecolor = "black")
    axs[2].set_xticks(x, labels)
    axs[2].set_title("Acc. 15$^\circ$")

    print(np.average(keras_acc_15), np.average(tflite_acc_15), np.average(qkeras_acc_15), np.std(keras_acc_15), np.std(tflite_acc_15), np.std(qkeras_acc_15))

    #fig.suptitle("TCN", fontsize=16, y=0.95)
    

In [ ]:
plot_metrics(results)